# 01 - Data Exploration

This notebook provides an interactive exploration of the PMU disturbance dataset.

**Contents:**
1. Load and inspect data structure
2. Basic statistics (PMUs, events, date ranges)
3. Data quality assessment
4. Initial visualizations

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from data_loader import load_pmu_disturbance_data, get_section_events, calculate_event_statistics

# Configuration
DATA_PATH = '../../data/PMU_disturbance.xlsx'
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

## 1. Load Data

In [ ]:
# Load PMU and Disturbance data
pmu_df, dist_df = load_pmu_disturbance_data(DATA_PATH)

print(f"PMU Records: {len(pmu_df)}")
print(f"Disturbance Events: {len(dist_df)}")
print(f"\nPMU Columns: {list(pmu_df.columns)}")
print(f"\nDisturbance Columns: {list(dist_df.columns)}")

In [ ]:
# Inspect PMU data
pmu_df.head(10)

In [ ]:
# Inspect Disturbance data
dist_df.head(10)

## 2. Basic Statistics

In [ ]:
# Date range
datetime_col = [c for c in dist_df.columns if 'time' in c.lower() or 'date' in c.lower()][0]
print(f"Date Range: {dist_df[datetime_col].min()} to {dist_df[datetime_col].max()}")

# Events per section
section_col = [c for c in dist_df.columns if 'section' in c.lower()][0]
events_per_section = dist_df.groupby(section_col).size()
print(f"\nEvents per Section:")
print(f"  Mean: {events_per_section.mean():.1f}")
print(f"  Median: {events_per_section.median():.1f}")
print(f"  Max: {events_per_section.max()} (Section {events_per_section.idxmax()})")
print(f"  Min: {events_per_section.min()}")

In [ ]:
# Distribution of events per section
fig, ax = plt.subplots(figsize=(12, 5))
events_per_section.hist(bins=50, ax=ax, color='steelblue', edgecolor='white')
ax.axvline(events_per_section.mean(), color='red', linestyle='--', label=f'Mean: {events_per_section.mean():.1f}')
ax.axvline(events_per_section.max(), color='orange', linestyle='--', label=f'Max: {events_per_section.max()}')
ax.set_xlabel('Events per Section')
ax.set_ylabel('Number of Sections')
ax.set_title('Distribution of Disturbance Events Across Sections')
ax.legend()
plt.tight_layout()
plt.show()

## 3. Data Quality Assessment

In [ ]:
# Missing values
print("Missing Values in PMU Data:")
print(pmu_df.isnull().sum())
print("\nMissing Values in Disturbance Data:")
print(dist_df.isnull().sum())

In [ ]:
# Data types
print("PMU Data Types:")
print(pmu_df.dtypes)
print("\nDisturbance Data Types:")
print(dist_df.dtypes)

## 4. Initial Visualizations

In [ ]:
# Top 20 sections by event count
top_sections = events_per_section.nlargest(20)

fig, ax = plt.subplots(figsize=(12, 6))
top_sections.plot(kind='bar', ax=ax, color='steelblue')
ax.set_xlabel('Section ID')
ax.set_ylabel('Event Count')
ax.set_title('Top 20 Sections by Disturbance Event Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Events over time
daily_events = dist_df.set_index(datetime_col).resample('M').size()

fig, ax = plt.subplots(figsize=(14, 5))
daily_events.plot(ax=ax, color='steelblue')
ax.set_xlabel('Date')
ax.set_ylabel('Monthly Event Count')
ax.set_title('Disturbance Events Over Time (Monthly)')
plt.tight_layout()
plt.show()

In [ ]:
# Cause distribution
cause_col = [c for c in dist_df.columns if 'cause' in c.lower()][0]
cause_counts = dist_df[cause_col].value_counts().head(15)

fig, ax = plt.subplots(figsize=(12, 6))
cause_counts.plot(kind='barh', ax=ax, color='steelblue')
ax.set_xlabel('Event Count')
ax.set_ylabel('Cause')
ax.set_title('Top 15 Disturbance Causes (Network-Wide)')
plt.tight_layout()
plt.show()

## Summary

Key findings from data exploration:
- **533 PMU sections** in the network
- **9,369 disturbance events** recorded
- Highly skewed distribution - most sections have few events, but some have hundreds
- Section 150 has the most events (301)
- Data quality appears good with minimal missing values